# Imports

In [31]:
import os
import folium
import openpyxl
import pandas as pd
import geopandas as gpd
import numpy as np

# Read and edit data from cars

In [21]:
autos = pd.read_csv('../Ausgabedateien/autos_aufbereitet.csv')

In [22]:
autos.head(2)

,Unnamed: 0,web-scraper-order,web-scraper-start-url,Preis,plz,jahr,ps-stand,Farbe,PLZ4,%_IN_GDE,KTKZ,GDENR,GDENAMK,Hist.-Nummer,Kanton,Bezirks-nummer,Bezirksname,BFS Gde-nummer,Gemeindename,Datum der Aufnahme
0,0,1669622809-189,https://www.autolina.ch/elektro-auto,49'650,8640,NaN,231.0,silber,8640.0,3.91,SZ,1322.0,Freienbach,11961.0,SZ,503.0,Höfe,1322,Freienbach,1848-09-12
1,1,1669622809-190,https://www.autolina.ch/elektro-auto,42'500,6464,2022.0,204.0,weiss,6464.0,0.36,UR,1205.0,Bürglen (UR),11506.0,UR,400.0,Kt. Uri,1205,Bürglen (UR),1848-09-12


In [23]:
autos = autos.astype({"BFS Gde-nummer":int})
autos.head(2)

,Unnamed: 0,web-scraper-order,web-scraper-start-url,Preis,plz,jahr,ps-stand,Farbe,PLZ4,%_IN_GDE,KTKZ,GDENR,GDENAMK,Hist.-Nummer,Kanton,Bezirks-nummer,Bezirksname,BFS Gde-nummer,Gemeindename,Datum der Aufnahme
0,0,1669622809-189,https://www.autolina.ch/elektro-auto,49'650,8640,NaN,231.0,silber,8640.0,3.91,SZ,1322.0,Freienbach,11961.0,SZ,503.0,Höfe,1322,Freienbach,1848-09-12
1,1,1669622809-190,https://www.autolina.ch/elektro-auto,42'500,6464,2022.0,204.0,weiss,6464.0,0.36,UR,1205.0,Bürglen (UR),11506.0,UR,400.0,Kt. Uri,1205,Bürglen (UR),1848-09-12


In [24]:
Angebote_pro_gemeinde = pd.DataFrame(autos['BFS Gde-nummer'].value_counts().rename_axis('BFS-Nummer').reset_index(name='Anzahl'))
Angebote_pro_gemeinde

,BFS-Nummer,Anzahl
0,4746,99
1,230,61
2,1059,54
3,4,49
4,3352,43
...,...,...
331,983,1
332,113,1
333,6708,1
334,84,1


In [25]:
Angebote_pro_gemeinde = Angebote_pro_gemeinde.astype({"Anzahl":int})
Angebote_pro_gemeinde = Angebote_pro_gemeinde.astype({"BFS-Nummer":int})
Angebote_pro_gemeinde.head(2)

,BFS-Nummer,Anzahl
0,4746,99
1,230,61


In [26]:
gemeinden = pd.read_csv('../Quelldateien/Gemeindestand.csv')

# Safe result to csv

In [27]:
Angebote_pro_gemeinde.to_csv('../Ausgabedateien/anzahl_autos_pro_gemeinde.csv')

# Read data of charging stations

In [28]:
stations = pd.read_csv('../Ausgabedateien/ladestellen_pro_gemeinde.csv')
print(stations.shape)
stations.head()

(1017, 3)


,Unnamed: 0,Postleitzahl,Anzahl
0,0,6340,97
1,1,-,71
2,2,6300,53
3,3,9015,51
4,4,6010,49


In [40]:
print(stations.dtypes)

Unnamed: 0       int64
Postleitzahl    object
Anzahl           int64
dtype: object


In [42]:
stations['Postleitzahl'].astype(str).astype(int)

ValueError: invalid literal for int() with base 10: '-'

In [39]:
stations.dropna(axis="Postleitzahl")

ValueError: No axis named Postleitzahl for object type DataFrame

In [32]:
plz = pd.read_csv('../Quelldateien/plz_gemeinde.csv') #csv with Names of "Gemeinden" and Postalcodes
plz['PLZ4'].replace('', np.nan, inplace=True)
plz['PLZ4'].replace('0', np.nan, inplace=True)
plz.dropna(subset=['PLZ4'], inplace=True)
plz['PLZ4']=plz['PLZ4'].astype(int)
plz.head(5)

,PLZ4,%_IN_GDE,KTKZ,GDENR,GDENAMK
0,8914,100.00,ZH,1.0,Aeugst am Albis
1,8909,14.34,ZH,2.0,Affoltern am Albis
2,8910,85.66,ZH,2.0,Affoltern am Albis
3,8906,100.00,ZH,3.0,Bonstetten
4,6340,0.77,ZH,4.0,Hausen am Albis


In [34]:
print(plz.shape)
plz = plz.drop_duplicates('PLZ4')
plz

(4917, 5)


,PLZ4,%_IN_GDE,KTKZ,GDENR,GDENAMK
0,8914,100.00,ZH,1.0,Aeugst am Albis
1,8909,14.34,ZH,2.0,Affoltern am Albis
2,8910,85.66,ZH,2.0,Affoltern am Albis
3,8906,100.00,ZH,3.0,Bonstetten
4,6340,0.77,ZH,4.0,Hausen am Albis
...,...,...,...,...,...
4910,2914,12.97,JU,6809.0,Haute-Ajoie
4913,2946,30.35,JU,6810.0,La Baroche
4914,2947,27.66,JU,6810.0,La Baroche
4915,2953,18.89,JU,6810.0,La Baroche


In [36]:
print(plz.shape)

(3184, 5)


In [35]:
stations_prepared = pd.merge(left=stations, right=plz, how='left', left_on='Postleitzahl', right_on='PLZ4')
print(stations_prepared.shape)
stations_prepared.head(5)

ValueError: You are trying to merge on object and int32 columns. If you wish to proceed you should use pd.concat

# Jupyter notebook --footer info

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
NT
Windows | 10
Datetime: 2022-12-26 17:15:30
Python Version: 3.10.1
-----------------------------------
